

## Lab 2 Pandas_Execise




# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Loading and Cleaning with Pandas</li>
<li> Parsing and Completing the Dataframe  </li>
<li> Grouping </li>
</ol>

## Learning Goals

About 6000 odd "best books" were fetched and parsed from [Goodreads](https://www.goodreads.com). The "bestness" of these books came from a proprietary formula used by Goodreads and published as a list on their web site.

We parsed the page for each book and saved data from all these pages in a tabular format as a CSV file. In this lab we'll clean and further parse the data.  We'll then do some exploratory data analysis to answer questions about these best books and popular genres.  


By the end of this lab, you should be able to:

- Load and systematically address missing values, ancoded as `NaN` values in our data set, for example, by removing observations associated with these values.
- Parse columns in the dataframe to create new dataframe columns.
- Use groupby to aggregate data on a particular feature column, such as author.

*This lab corresponds to lectures 2, 3 and 4 and maps on to homework 1 and further.*

### Basic EDA workflow

(From the lecture, repeated here for convenience).

The basic workflow is as follows:

1. **Build** a DataFrame from the data (ideally, put all data in this object)
2. **Clean** the DataFrame. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby and small multiples to compare subsets of the data.

This process transforms your data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to followup in subsequent analysis.

## Part 1: Loading and Cleaning with Pandas 
Read in the `goodreads.csv` file, examine the data, and do any necessary data cleaning. 

Here is a description of the columns (in order) present in this csv file:

```
rating: the average rating on a 1-5 scale achieved by the book
review_count: the number of Goodreads users who reviewed this book
isbn: the ISBN code for the book
booktype: an internal Goodreads identifier for the book
author_url: the Goodreads (relative) URL for the author of the book
year: the year the book was published
genre_urls: a string with '|' separated relative URLS of Goodreads genre pages
dir: a directory identifier internal to the scraping code
rating_count: the number of ratings for this book (this is different from the number of reviews)
name: the name of the book
```

Let us see what issues we find with the data and resolve them.  



----




After loading appropriate libraries


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

### Cleaning: Reading in the data
We read in and clean the data from `goodreads.csv`.

In [2]:
#Read the data into a dataframe
df = pd.read_csv("./data/goodreads.csv")

#Examine the first few rows of the dataframe
df.head()

,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.Suzanne_Collins,2008,/genres/young-adult|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|/genres/book-club|/genres/young-adult|/genres/teen|/genres/apocalyptic|/genres/post-apocalyptic|/genres/action,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)"
0,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
1,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
2,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
3,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice
4,4.25,12363,0446675539,good_reads:book,https://www.goodreads.com/author/show/11081.Ma...,1936.0,/genres/classics|/genres/historical-fiction|/g...,dir01/18405.Gone_with_the_Wind.html,645470,Gone with the Wind


Oh dear. That does not quite seem to be right. We are missing the column names. We need to add these in! But what are they?

Here is a list of them in order:

`["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']`

<div class="exercise"><b>Exercise</b></div>
Use these to load the dataframe properly! And then "head" the dataframe... (you will need to look at the read_csv docs)


In [3]:
# your code here
df.columns = ["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 
              'genre_urls', 'dir','rating_count', 'name']

#Examine the first few rows of the dataframe
df.head()

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
0,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
1,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
2,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
3,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice
4,4.25,12363,0446675539,good_reads:book,https://www.goodreads.com/author/show/11081.Ma...,1936.0,/genres/classics|/genres/historical-fiction|/g...,dir01/18405.Gone_with_the_Wind.html,645470,Gone with the Wind


### Cleaning: Examing the dataframe - quick checks

We should examine the dataframe to get a overall sense of the content. 

<div class="exercise"><b>Exercise</b></div>
Lets check the types of the columns. What do you find?

In [4]:
# your code here
####### 
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

*your answer here*

Notice that `review_count` and `rating_counts` are objects instead of ints, and the `year` is a float!

There are a couple more quick sanity checks to perform on the dataframe. 

In [5]:
print(df.shape)
df.columns

(5999, 10)


Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'genre_urls', 'dir', 'rating_count', 'name'], dtype='object')

### Cleaning: Examining the dataframe - a deeper look

Beyond performing checking some quick general properties of the data frame and looking at the first $n$ rows, we can dig a bit deeper into the values being stored. If you haven't already, check to see if there are any missing values in the data frame.

Let's see for a column which seemed OK to us.

In [6]:
# Get a sense of how many missing values there are in the dataframe.
np.sum([df.rating.isnull()])

2

In [7]:
# Try to locate where the missing values occur
df[df.rating.isnull()]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
3642,NaN,None,None,None,None,NaN,NaN,dir37/9658936-harry-potter.html,None,None
5281,NaN,None,None,None,None,NaN,NaN,dir53/113138.The_Winner.html,None,None


How does `pandas` or `numpy` handle missing values when we try to compute with data sets that include them?

We'll now check if any of the other suspicious columns have missing values.  Let's look at `year` and `review_count` first.

One thing you can do is to try and convert to the type you expect the column to be. If something goes wrong, it likely means your data are bad.

Lets test for missing data:

In [8]:
df[df.year.isnull()]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
2441,4.23,526,NaN,good_reads:book,https://www.goodreads.com/author/show/623606.A...,NaN,/genres/religion|/genres/islam|/genres/non-fic...,dir25/1301625.La_Tahzan.html,4134,La Tahzan
2868,4.61,2,NaN,good_reads:book,https://www.goodreads.com/author/show/8182217....,NaN,NaN,dir29/22031070-my-death-experiences---a-preach...,23,My Death Experiences - A Preacherâs 18 Apoca...
3642,NaN,None,None,None,None,NaN,NaN,dir37/9658936-harry-potter.html,None,None
5281,NaN,None,None,None,None,NaN,NaN,dir53/113138.The_Winner.html,None,None
5571,3.71,35,8423336603,good_reads:book,https://www.goodreads.com/author/show/285658.E...,NaN,/genres/fiction,dir56/890680._rase_una_vez_el_amor_pero_tuve_q...,403,Ãrase una vez el amor pero tuve que matarlo. ...
5657,4.32,44,NaN,good_reads:book,https://www.goodreads.com/author/show/25307.Ro...,NaN,/genres/fantasy|/genres/fantasy|/genres/epic-f...,dir57/5533041-assassin-s-apprentice-royal-assa...,3850,Assassin's Apprentice / Royal Assassin (Farsee...
5682,4.56,204,NaN,good_reads:book,https://www.goodreads.com/author/show/3097905....,NaN,/genres/fantasy|/genres/young-adult|/genres/ro...,dir57/12474623-tiger-s-dream.html,895,"Tiger's Dream (The Tiger Saga, #5)"


### Cleaning: Dealing with Missing Values
How should we interpret 'missing' or 'invalid' values in the data (hint: look at where these values occur)? One approach is to simply exclude them from the dataframe. Is this appropriate for all 'missing' or 'invalid' values? 

In [9]:
# Treat the missing or invalid values in your dataframe
####### 
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
np.sum(df.isnull())

rating          0
review_count    0
isbn            0
booktype        0
author_url      0
year            0
genre_urls      0
dir             0
rating_count    0
name            0
dtype: int64

Ok so we have done some cleaning. What do things look like now? Notice the float has not yet changed.

In [10]:
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

In [11]:
print(np.sum(df.year.isnull()))
print(np.sum(df.rating_count.isnull())) 
print(np.sum(df.review_count.isnull())) 
# We removed seven rows
df.shape

0
0
0


(5495, 10)

*Suspect observations for rating and rating_count were removed as well! *

<div class="exercise"><b>Exercise</b></div>

Ok so lets fix those types. Convert them to ints. If the type conversion fails, we now know we have further problems.

In [12]:
# your code here
df['rating'] = df['rating'].astype("int")
df['rating_count'] = df['rating_count'].astype("int")

Once you do this, we seem to be good on these columns (no errors in conversion). Lets look:

In [13]:
df.dtypes

rating            int32
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count      int32
name             object
dtype: object

Sweet!

Some of the other colums that should be strings have NaN.

In [14]:
df.loc[df.genre_urls.isnull(), 'genre_urls']=""
df.loc[df.isbn.isnull(), 'isbn']=""

##  Part 2: Parsing and Completing the Data Frame 

We will parse the `author` column from the author_url and `genres` column from the genre_urls. Keep the `genres` column as a string separated by '|'.

We will use panda's `map` to assign new columns to the dataframe.  

Examine an example `author_url` and reason about which sequence of string operations must be performed in order to isolate the author's name.

In [15]:
#Get the first author_url
test_string = df.author_url[0]
test_string

'https://www.goodreads.com/author/show/1077326.J_K_Rowling'

In [16]:
#Test out some string operations to isolate the author name

test_string.split('/')[-1].split('.')[1:][0]

'J_K_Rowling'

<div class="exercise"><b>Exercise</b></div>

Lets wrap the above code into a function which we will then use

In [17]:
# Write a function that accepts an author url and returns the author's name based on your experimentation above
def get_author(url):
    # your code here
    name = url.split('/')[-1].split('.')[1:][0]
    ####### 
    return name

In [18]:
#Apply the get_author function to the 'author_url' column using '.map' 
#and add a new column 'author' to store the names
df['author'] = df.author_url.map(get_author)
df.author[0:5]

0          J_K_Rowling
1      Stephenie_Meyer
2           Harper_Lee
3          Jane_Austen
4    Margaret_Mitchell
Name: author, dtype: object

<div class="exercise"><b>Exercise</b></div>

Now parse out the genres from `genre_url`.  

This is a little more complicated because there be more than one genre.


In [19]:
df.genre_urls.head()

0    /genres/fantasy|/genres/young-adult|/genres/fi...
1    /genres/young-adult|/genres/fantasy|/genres/ro...
2    /genres/classics|/genres/fiction|/genres/histo...
3    /genres/classics|/genres/fiction|/genres/roman...
4    /genres/classics|/genres/historical-fiction|/g...
Name: genre_urls, dtype: object

In [20]:
# your code here
# Examine some examples of genre_urls
def getGenre(url):
    genre = '|'.join(j for j in [i.split('/')[2] for i in url.split('|')])
    
    return genre
# Test out some string operations to isolate the genre name
g = getGenre(df['genre_urls'][0])
g

'fantasy|young-adult|fiction|fantasy|magic|childrens|adventure|science-fiction-fantasy'

<div class="exercise"><b>Exercise</b></div>

Write a function that accepts a genre url and returns the genre name based on your experimentation above



In [21]:
def split_and_join_genres(url):
    # your code here
    if url == '':
        return ''
    else:
        return getGenre(url)

Test your function

In [22]:
split_and_join_genres("/genres/young-adult|/genres/science-fiction")

'young-adult|science-fiction'

In [23]:
split_and_join_genres("")

''

<div class="exercise"><b>Exercise</b></div>

Use map again to create a new "genres" column

In [24]:
df['genres']=df.genre_urls.map(split_and_join_genres)
df.head()

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
0,4,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...,J_K_Rowling,fantasy|young-adult|fiction|fantasy|magic|chil...
1,3,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)",Stephenie_Meyer,young-adult|fantasy|romance|paranormal|vampire...
2,4,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird,Harper_Lee,classics|fiction|historical-fiction|academic|s...
3,4,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice,Jane_Austen,classics|fiction|romance|historical-fiction|li...
4,4,12363,0446675539,good_reads:book,https://www.goodreads.com/author/show/11081.Ma...,1936.0,/genres/classics|/genres/historical-fiction|/g...,dir01/18405.Gone_with_the_Wind.html,645470,Gone with the Wind,Margaret_Mitchell,classics|historical-fiction|fiction|romance|li...


Finally, let's pick an author at random so we can see the results of the transformations.  Scroll to see the `author` and `genre` columns that we added to the dataframe.

In [25]:
df[df.author == "Marguerite_Yourcenar"]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
1013,4,483,0374529264,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1951.0,/genres/historical-fiction|/genres/fiction|/ge...,dir11/12172.Memoirs_of_Hadrian.html,6258,Memoirs of Hadrian,Marguerite_Yourcenar,historical-fiction|fiction|cultural|france|cla...
5619,4,74,2070367983,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1968.0,/genres/fiction|/genres/historical-fiction|/ge...,dir57/953435.L_uvre_au_noir.html,1601,L'Åuvre au noir,Marguerite_Yourcenar,fiction|historical-fiction|cultural|france|eur...


Let us delete the `genre_urls` column.

In [26]:
del df['genre_urls']

And then save the dataframe out!

In [27]:
df.to_csv("data/cleaned-goodreads.csv", index=False, header=True)

---

## Part 3: Grouping 

It appears that some books were written in negative years!  Print out the observations that correspond to negative years.  What do you notice about these books?  

In [28]:
# your code here
df[df.year < 0]
# These are books written before the Common Era (BCE, equivalent to BC).

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,author,genres
46,3,5785,0143039954,good_reads:book,https://www.goodreads.com/author/show/903.Homer,-800.0,dir01/1381.The_Odyssey.html,560248,The Odyssey,Homer,classics|fiction|poetry|fantasy|mythology|acad...
245,4,365,0147712556,good_reads:book,https://www.goodreads.com/author/show/903.Homer,-800.0,dir03/1375.The_Iliad_The_Odyssey.html,35123,The Iliad/The Odyssey,Homer,classics|fantasy|mythology|fantasy|academic|sc...
454,3,1499,0140449140,good_reads:book,https://www.goodreads.com/author/show/879.Plato,-380.0,dir05/30289.The_Republic.html,82022,The Republic,Plato,philosophy|classics|non-fiction|politics|histo...
595,3,1240,0679729526,good_reads:book,https://www.goodreads.com/author/show/919.Virgil,-29.0,dir06/12914.The_Aeneid.html,60308,The Aeneid,Virgil,classics|poetry|fiction|fantasy|mythology|acad...
628,3,1231,1580495931,good_reads:book,https://www.goodreads.com/author/show/1002.Sop...,-429.0,dir07/1554.Oedipus_Rex.html,93192,Oedipus Rex,Sophocles,classics|plays|drama|fiction|academic|school|l...
673,3,3559,1590302257,good_reads:book,https://www.goodreads.com/author/show/1771.Sun...,-512.0,dir07/10534.The_Art_of_War.html,114619,The Art of War,Sun_Tzu,non-fiction|politics|classics|literature|psych...
745,4,1087,0140449183,good_reads:book,https://www.goodreads.com/author/show/5158478....,-500.0,dir08/99944.The_Bhagavad_Gita.html,31634,The Bhagavad Gita,Anonymous,classics|spirituality|religion|hinduism|fantas...
776,3,1038,1580493882,good_reads:book,https://www.goodreads.com/author/show/1002.Sop...,-442.0,dir08/7728.Antigone.html,49084,Antigone,Sophocles,drama|fiction|classics|academic|read-for-schoo...
1232,3,704,015602764X,good_reads:book,https://www.goodreads.com/author/show/1002.Sop...,-400.0,dir13/1540.The_Oedipus_Cycle.html,36008,The Oedipus Cycle,Sophocles,classics|plays|drama|fiction|literature|academ...
1396,4,890,0192840509,good_reads:book,https://www.goodreads.com/author/show/12452.Aesop,-560.0,dir14/21348.Aesop_s_Fables.html,71259,Aesop's Fables,Aesop,classics|childrens|literature|fantasy|fairy-ta...


We can determine the "best book" by year! For this we use Pandas groupby. Groupby allows grouping a dataframe by any (usually categorical) variable.

In [29]:
dfgb_author = df.groupby('author')
type(dfgb_author)

pandas.core.groupby.generic.DataFrameGroupBy

Perhaps we want the number of books each author wrote

In [30]:
dfgb_author.count()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,genres
author,,,,,,,,,,
A_A_Milne,6,6,6,6,6,6,6,6,6,6
A_G_Howard,1,1,1,1,1,1,1,1,1,1
A_J_Cronin,1,1,1,1,1,1,1,1,1,1
A_J_Jacobs,1,1,1,1,1,1,1,1,1,1
A_N_Roquelaure,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...
_,7,7,7,7,7,7,7,7,7,7
_gota_Krist_f,1,1,1,1,1,1,1,1,1,1
_mile_Zola,4,4,4,4,4,4,4,4,4,4


Lots of useless info there. One column should suffice

In [31]:
dfgb_author['author'].count()

author
A_A_Milne                6
A_G_Howard               1
A_J_Cronin               1
A_J_Jacobs               1
A_N_Roquelaure           2
                        ..
_                        7
_gota_Krist_f            1
_mile_Zola               4
_ric_Emmanuel_Schmitt    1
_sne_Seierstad           1
Name: author, Length: 2427, dtype: int64

Perhaps you want more detailed info...

In [32]:
dfgb_author[['rating', 'rating_count', 'review_count', 'year']].describe()

rating                                               rating_count                                                                              year                                                                   
                       count      mean       std  min   25%  50%  75%  max        count          mean           std      min       25%      50%       75%       max count         mean         std     min      25%     50%      75%     max
author                                                                                                                                                                                                                                      
A_A_Milne                6.0  4.000000  0.000000  4.0  4.00  4.0  4.0  4.0          6.0  47842.333333  57135.314010    544.0  15559.50  30547.0  50401.25  157833.0   6.0  1944.166667   29.294482  1926.0  1926.25  1927.5  1952.75  1997.0
A_G_Howard               1.0  4.000000       NaN  4.0  4.00  4.0  4.0  4.0          1.0  17073.000000           NaN  17073.0  17073.00  17073.0  17073.00   17073.0   1.0  2013.000000         NaN  2013.0  2013.00  2013.0  2013.00  2013.0
A_J_Cronin               1.0  4.000000       NaN  4.0  4.00  4.0  4.0  4.0          1.0   1015.000000           NaN   1015.0   1015.00   1015.0   1015.00    1015.0   1.0  1941.000000         NaN  1941.0  1941.00  1941.0  1941.00  1941.0
A_J_Jacobs               1.0  3.000000       NaN  3.0  3.00  3.0  3.0  3.0          1.0  39489.000000           NaN  39489.0  39489.00  39489.0  39489.00   39489.0   1.0  2007.000000         NaN  2007.0  2007.00  2007.0  2007.00  2007.0
A_N_Roquelaure           2.0  3.000000  0.000000  3.0  3.00  3.0  3.0  3.0          2.0  25967.000000   9480.887722  19263.0  22615.00  25967.0  29319.00   32671.0   2.0  1983.500000    0.707107  1983.0  1983.25  1983.5  1983.75  1984.0
...                      ...       ...       ...  ...   ...  ...  ...  ...          ...           ...           ...      ...       ...      ...       ...       ...   ...          ...         ...     ...      ...     ...      ...     ...
_                        7.0  3.571429  0.534522  3.0  3.00  4.0  4.0  4.0          7.0  11557.000000   5873.244986   4540.0   7164.00  11407.0  14675.50   21273.0   7.0  1991.142857   18.433923  1963.0  1979.50  1994.0  2007.00  2008.0
_gota_Krist_f            1.0  4.000000       NaN  4.0  4.00  4.0  4.0  4.0          1.0   2628.000000           NaN   2628.0   2628.00   2628.0   2628.00    2628.0   1.0  1986.000000         NaN  1986.0  1986.00  1986.0  1986.00  1986.0
_mile_Zola               4.0  3.750000  0.500000  3.0  3.75  4.0  4.0  4.0          4.0   8358.250000   4553.226942   3594.0   5181.75   8095.0  11271.50   13649.0   4.0  1652.500000  456.345264   968.0  1649.75  1878.5  1881.25  1885.0
_ric_Emmanuel_Schmitt    1.0  4.000000       NaN  4.0  4.00  4.0  4.0  4.0          1.0   6439.000000           NaN   6439.0   6439.00   6439.0   6439.00    6439.0   1.0  2002.000000         NaN  2002.0  2002.00  2002.0  2002.00  2002.0
_sne_Seierstad           1.0  3.000000       NaN  3.0  3.00  3.0  3.0  3.0          1.0  24993.000000           NaN  24993.0  24993.00  24993.0  24993.00   24993.0   1.0  2002.000000         NaN  2002.0  2002.00  2002.0  2002.00  2002.0

[2427 rows x 24 columns]

You can also access a `groupby` dictionary style.

In [33]:
ratingdict = {}
for author, subset in dfgb_author:
    ratingdict[author] = (subset['rating'].mean(), subset['rating'].std())
# ratingdict

<div class="exercise"><b>Exercise</b></div>

Lets get the best-rated book(s) for every year in our dataframe.

In [101]:
# Using .groupby, we can divide the dataframe into subsets by the values of 'year'.
# We can then iterate over these subsets
# your code here
df_year = df.groupby('year')
df_year[['rating', 'name']].max()

,rating,name
year,,
-1500.0,3,The Epic of Gilgamesh
-800.0,4,The Odyssey
-560.0,4,Aesop's Fables
-512.0,3,The Art of War
-500.0,4,The Bhagavad Gita
...,...,...
2010.0,4,Zombies Vs. Unicorns
2011.0,4,Wonderstruck
2012.0,4,"Wrath (New Species, #6)"
